In [42]:
import os
import glob
import time
import pandas as pd  # Import pandas for data manipulation

import PIL
import imageio
import numpy as np
import tensorflow as tf
from PIL import Image
from tensorflow.keras import layers
from IPython import display
import matplotlib.pyplot as plt
%matplotlib inline

print('tensorflow', tf.__version__)

tensorflow 2.12.0


In [43]:
# Load and preprocess the composition data
composition_data = pd.read_excel('./Al_data/7000_Al.xlsx', header=0, sheet_name="Sheet3")
composition_data = composition_data.drop('class', axis=1)
composition_data.fillna(0, inplace=True)
composition_data = composition_data.apply(pd.to_numeric, errors='coerce')


# 각 열별 최대값과 최소값 계산
composition_max = composition_data.max(axis=0)
composition_min = composition_data.min(axis=0)

# 조성 데이터 정규화
composition_data_normalized = (composition_data - composition_min) / (composition_max - composition_min) * 2 - 1

In [44]:
# Load and preprocess the image data
image_paths = glob.glob('./Al_data/images/*.*')
image_data = []
for path in image_paths:
    img = Image.open(path)
    img = img.resize((28, 28))
    img = (np.array(img) - 127.5) / 127.5
    image_data.append(img)
image_data = np.array(image_data)
image_data = image_data[:, :, :, 0:1]
image_data = image_data.astype('float32')

In [53]:
print(composition_data_normalized.shape)
print(image_data.shape)

(69, 12)
(69, 28, 28, 1)


In [48]:
combined_data = []
for i in range(len(composition_data)):
    composition_vector = composition_data_normalized[i]  # 정규화된 조성 데이터
    image = image_data[i]  # 이미지 데이터
    combined_data.append((composition_vector, image))

combined_data = np.array(combined_data)


KeyError: 0